In [ ]:
!pip install -q -U transformers[torch]
!pip install -q -U datasets
!pip install -q -U evaluate
!pip install -q -U tokenizers
!pip install -q -U bitsandbytes
!pip install -q rouge_score
!pip install -q -U peft
!pip install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [ ]:
import re
import random
import numpy as np
from scipy.special import softmax
import pprint

import bitsandbytes as bnb

import torch
import transformers
import evaluate
from datasets import Dataset, load_dataset

# For from-scratch T5 model
from transformers import T5TokenizerFast, T5Config, T5ForConditionalGeneration

# For pre-trained T5 model
from transformers import T5Tokenizer, T5ForConditionalGeneration  # this won't import twice, just noting here what's for each model

# For all T5 models
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# For BLEURT (to load a trained model for evaluation)
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM


# For style classifier model (also for evaluating the seq2seq model output)
from transformers import BertTokenizer, BertForSequenceClassification, BitsAndBytesConfig,pipeline
from transformers import TrainingArguments, Trainer

import pandas as pd
from tqdm import tqdm

# Local Data Loading Load Data

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to data save in Drive
train = 'FairytaleQA_train.csv'
valid = 'FairytaleQA_valid.csv'
test = 'FairytaleQA_test.csv'
path = 'drive/MyDrive/266_Danielle_Dylan_final_project/data/'
# path = 'drive/MyDrive/DataSci_266/266_Danielle_Dylan_final_project/data/'

In [ ]:
train_filepath = path+train
valid_filepath = path+valid
test_filepath = path+test

# train = pd.read_csv(train)
# valid = pd.read_csv(valid)
# test = pd.read_csv(test)

In [ ]:
# Save this because we'll need to tell the trainer how many examples we have
num_train_examples = 8_548

In [ ]:
train = load_dataset("csv",data_files=train_filepath,streaming=True)['train']
val = load_dataset("csv",data_files=valid_filepath,streaming=True)['train']
test = load_dataset("csv",data_files=test_filepath,streaming=True)['train']

In [ ]:
train

IterableDataset({
    features: Unknown,
    num_shards: 1
})

In [ ]:
from datasets import load_dataset, DatasetDict
# train_ds = Dataset.from_pandas(train, split="train")
# test_ds = Dataset.from_pandas(test, split="test")
# valid_ds = Dataset.from_pandas(valid, split="valid")

# Combine into a single DatasetDict
ds = DatasetDict({
    "train": train,
    "test": test,
    "validation": val,
})

In [ ]:
train = ds['train'].shuffle()
val = ds['validation'].shuffle()
test = ds['test'].shuffle()

In [ ]:
"""
Initialize the pipeline with bitsandbytes quantization
"""
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
VOCAB_SIZE = 15000
MAX_SEQUENCE_LENGTH = 512

In [ ]:
model_checkpoint = 'Qwen/Qwen2-0.5B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question with one sentence: "
context = " Context: "

# Define the preprocessing function

def preprocess_function(data):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   data['full_input'] = [context + story + prefix + question for question,story in zip(data["question"],data['story_section'])]

   input_encoded = tokenizer.batch_encode_plus(data['full_input'],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   answers_encoded = tokenizer.batch_encode_plus(data["answer1"],
                      max_length=MAX_SEQUENCE_LENGTH,
                      truncation=True,
                      padding='max_length',
                      return_tensors='pt')

   return {'input_ids': input_encoded['input_ids'],
           'labels': answers_encoded['input_ids']}

In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question with one sentence."
context = " Context: "

# Define the preprocessing function

def preprocess_function_qwen_chat(data):
    """Preprocess data using Qwen2 chat format"""

    full_inputs = []
    for question, story, answer in zip(data["question"], data['story_section'], data['answer1']):
        # Format as a conversation
        messages = [
            {"role": "user", "content": f"Context: {story} {prefix}\n\nQuestion: {question}"},
            {"role": "assistant", "content": answer}
        ]

        # Apply chat template
        formatted_text = tokenizer.apply_chat_template(messages, tokenize=False)
        formatted_text = f"User: Context: {story}\n\nQuestion: {question}\nAssistant: {answer}"
        # print(formatted_text)
        full_inputs.append(formatted_text)

    # Tokenize
    tokenized = tokenizer(
        full_inputs,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        padding='max_length'
    )

    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

In [ ]:
def create_seq2seq_training_args(batch_size, num_epochs):

    training_args = Seq2SeqTrainingArguments(
        "fairytale_QA_model",
        eval_strategy='epoch',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        report_to='none'

    )

    return training_args

In [ ]:
def create_seq2seq_trainer(model, training_args, train_ds, val_ds):

    trainer = Seq2SeqTrainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

In [ ]:
def create_training_args(batch_size, num_epochs):

    training_args = TrainingArguments(
        "fairytale_QA_model",
        eval_strategy='epoch',
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        report_to='none'

    )

    return training_args

In [ ]:
def create_trainer(model, training_args, train_ds, val_ds):

    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds
    )

    return trainer

# Fine Tuning Together

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
# Load the BLEU metric and the trained BLEURT model for semantic similarity scoring

bleu = evaluate.load("bleu")

bleurt_tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
bleurt_model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

# Trying Llama Family on Implicit questions

In [ ]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import torch
from transformers import AutoTokenizer

In [ ]:
num_train_examples_im = 2_166

In [ ]:
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
rouge = evaluate.load('rouge')

def compute_metrics_rouge(p):
  predictions, labels = p
  return rouge.compute(predictions=predictions, references=labels)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# model_checkpoint = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_checkpoint = 'Qwen/Qwen2-1.5B-Instruct'
model_checkpoint = 'Qwen/Qwen2-0.5B-Instruct'
# model_checkpoint = 'Qwen/Qwen2-0.5B'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

lr = 1e-5
batch_size = 8
num_epochs = 3

In [ ]:
# set LORA specific hyperparameters
r = 8
lora_alpha = 32
lora_dropout = 0.1

In [ ]:
model_im_qwen = AutoModelForCausalLM.from_pretrained(model_checkpoint,quantization_config=quantization_config,
                                                      device_map='auto')

# for name, param in model.named_parameters():
#     param.requires_grad = False

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)

In [ ]:
model_im_qwen = get_peft_model(model_im_qwen, peft_config)
model_im_qwen.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [ ]:
_= model_im_qwen.to("cuda")

In [ ]:
# Map the preprocessing function across our dataset
train_im = train.filter(lambda x: x['ex_or_im'] == 'implicit')
val_im = val.filter(lambda x: x['ex_or_im'] == 'implicit')
test_im = test.filter(lambda x: x['ex_or_im'] == 'implicit')

train_tokenized_im = train_im.map(preprocess_function_qwen_chat, batched=True)
val_tokenized_im = val_im.map(preprocess_function_qwen_chat, batched=True)
test_tokenized_im = test_im.map(preprocess_function_qwen_chat, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="qwen-small-lora-text-generation",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    max_steps=int(num_epochs * num_train_examples_im / batch_size),
    report_to='none'
)

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer = Trainer(
    model=model_im_qwen,
    args=training_args,
    train_dataset=train_tokenized_im,
    eval_dataset=val_tokenized_im,
    #compute_metrics=compute_metrics_rouge,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.653000,1.638147
1,1.416300,1.583920
2,1.399500,1.576967


TrainOutput(global_step=812, training_loss=2.231583584118359, metrics={'train_runtime': 2656.7356, 'train_samples_per_second': 2.445, 'train_steps_per_second': 0.306, 'total_flos': 7148496647356416.0, 'train_loss': 2.231583584118359, 'epoch': 2.332512315270936})

In [ ]:
model_qwen_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/qwen_model_im_chat'
model_im_qwen.save_pretrained(model_qwen_checkpoint_filepath, from_pt=True)

In [ ]:
# Run this line only if you need to reload the model you trained earlier
model_qwen_checkpoint_filepath = 'drive/MyDrive/266_Danielle_Dylan_final_project/checkpoints/qwen_model_im_chat'
model_im_qwen = AutoModelForCausalLM.from_pretrained(model_qwen_checkpoint_filepath)

In [ ]:
# pipe = pipeline('text-generation',model=model_im_qwen,tokenizer =tokenizer,max_new_tokens=200)

In [ ]:
def generate_qwen_output(model,tokenizer,text):


  messages = [
      {"role": "user", "content": text},
      {"role": "assistant", "content": ""},
       {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant designed to answer questions on a story."},
  ]
  text = tokenizer.apply_chat_template(
      messages[:-1],
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=100,
      do_sample=False,
      pad_token_id=tokenizer.eos_token_id
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return response

In [ ]:
def test_time_scaled_generation(model, tokenizer, story, question, steps=2, max_tokens_per_step=10):
    delimiter = "\nFinal Answer:"

    input_prompt = f"""Story:\n{story}\n\nQuestion: {question}\nWhat is the answer to the question based on the story?{delimiter}"""
    text = input_prompt

    for _ in range(steps):
        decoded = generate_qwen_output(model, tokenizer, text)
        if delimiter in decoded:
            text += " Wait."
        else:
            text = decoded

    final_inputs = tokenizer(text + f" {delimiter}", return_tensors="pt").to(model.device)
    final_output = model.generate(
        **final_inputs,
        max_new_tokens=max_tokens_per_step,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(final_output[0], skip_special_tokens=True)

    # # Extract just the answer portion
    # if "Final Answer:" in decoded:
    #     answer = decoded.split("Final Answer:")[-1].strip()
    # else:
    #     answer = decoded.strip()

    return decoded


In [ ]:
test_im_ds = Dataset.from_list(list(test_im))
test_im_ds

Dataset({
    features: ['story_name', 'story_section', 'question', 'answer1', 'answer2', 'local_or_sum', 'attribute', 'ex_or_im', 'ex_or_im2'],
    num_rows: 253
})

In [ ]:
question = context + test_im_ds['story_section'][0] + prefix + test_im_ds['question'][0]
question

" Context: he was always accompanied on these occasions by an enormous grey - headed raven , who lived in the cave along with him , and who was his bosom friend and companion . the islanders feared this bird of ill omen as much , perhaps , as they feared its master ; for , although they went to consult snorro in all their difficulties and perplexities , and bought medicines and love - potions from him , they always looked upon him with a certain dread , feeling that there was something weird and uncanny about him . now , at the time we are speaking of , orkney was governed by two earls , who were half - brothers . paul , the elder , was a tall , handsome man , with dark hair , and eyes like sloes . all the country people loved him , for he was so skilled in knightly exercises , and had such a sweet and loving nature , that no one could help being fond of him . old people 's eyes would brighten at the sight of him , and the little children would run out to greet him as he rode by their 

In [ ]:
test_time_scaled_generation(model_im_qwen,tokenizer,test_im_ds['story_section'][0],test_im_ds['question'][0])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


"The raven is indeed considered a bad omen in many cultures around the world. It is believed that the bird can bring misfortune or harm to those who encounter it. The islanders were therefore afraid of the raven's presence and its ability to fly away from their homes.\n\nIt is important to note that this fear may not always be justified. While the raven may bring misfortune or harm to those who encounter it, it is also possible for the raven to bring good fortune or \nFinal Answer: prosperity. In fact, the raven has been"

In [ ]:
test_im_ds['question'][0]

'why did the islanders fear the raven ?'

In [ ]:
test_im_ds['story_section'][0]

"he was always accompanied on these occasions by an enormous grey - headed raven , who lived in the cave along with him , and who was his bosom friend and companion . the islanders feared this bird of ill omen as much , perhaps , as they feared its master ; for , although they went to consult snorro in all their difficulties and perplexities , and bought medicines and love - potions from him , they always looked upon him with a certain dread , feeling that there was something weird and uncanny about him . now , at the time we are speaking of , orkney was governed by two earls , who were half - brothers . paul , the elder , was a tall , handsome man , with dark hair , and eyes like sloes . all the country people loved him , for he was so skilled in knightly exercises , and had such a sweet and loving nature , that no one could help being fond of him . old people 's eyes would brighten at the sight of him , and the little children would run out to greet him as he rode by their mothers ' 

In [ ]:
len(question)

1269

In [ ]:
test_im_ds['answer2'][0]

'he was a bird of ill omen .'

In [ ]:
test_im_ds['answer1'][0]

'it was a bad omen .'

In [ ]:
%%time

# Store results for aggregate scoring

def process_dataset(dataset):
  results = []
  bleurt_scores = []
  for idx, sample in enumerate(tqdm(dataset)):

      # if idx >= 10:  # Stop after processing 10 samples
      #   return pd.DataFrame(results)
      #   break

      prefix = "Please answer this question with one sentence: "
      context = " Context: "

      question = context + sample['story_section'] + prefix + sample['question']

      # Generate summary via the pipeline
      # outputs = pipe(
      #                     question,
      #                     max_new_tokens=50,
      #                     truncation=True,
      #                     temperature=0.6
      # )


      # answer = outputs[0]["generated_text"][len(question):]
      answer = generate_qwen_output(model_im_qwen,tokenizer,sample['story_section'],sample['question'])
      # print(len(answer))
      if (answer.isspace()) | (len(answer) < 1):
        print("\nANSWER IS SPACE for question:",sample['question'],"\nReference:", sample['answer1'])
        print("\nanswer:",[answer])
        answer = 'blank'

      # print("\nbegin answer:", answer, ":end answer")
      # print("\n",[answer],"****")

      # Evaluation
      predictions = [answer]
      reference1 = [sample['answer1']]
      reference2 = [sample['answer2']]

      # Calculate ROUGE scores
      rouge_scores1 = rouge.compute(predictions=predictions, references=reference1)
      if reference1 != reference2:
        rouge_scores2 = rouge.compute(predictions=predictions, references=reference2)
      else:
        rouge_scores2 = rouge_scores1
      rouge_scores1 = {k+'_a1': v for k, v in rouge_scores1.items()}
      rouge_scores2 = {k+'_a2': v for k, v in rouge_scores2.items()}


      # Calculate BLEU scores
      bleu_scores1 = bleu.compute(predictions=predictions, references=reference1,smooth=True)
      if reference1 != reference2:
        bleu_scores2 = bleu.compute(predictions=predictions, references=reference2,smooth=True)
      else:
        bleu_scores2 = bleu_scores1
      bleu_scores1 = {k+'_a1': v for k, v in bleu_scores1.items()}
      bleu_scores2 = {k+'_a2': v for k, v in bleu_scores2.items()}

      # Calculate BLEURT scores
      with torch.no_grad():
        bleurt_score_a1 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer1'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        if reference1 != reference2:
          bleurt_score_a2 = bleurt_model(**bleurt_tokenizer(answer,
                                                sample['answer2'],
                                                truncation=True,
                                                max_length=512,
                                                padding='max_length',
                                                return_tensors='pt'))[0].squeeze().numpy()
        else:
          bleurt_score_a2 = bleurt_score_a1


      # Store results
      results.append({
          'id': idx,
          'question': sample['question'],
          'ex_or_im': sample['ex_or_im'],
          'story_section': sample['story_section'][:500],  # Store truncated text for readability
          'reference_answer1': sample['answer1'],
          'reference_answer2': sample['answer2'],
          'generated_answer': answer,
          **rouge_scores1,
          **rouge_scores2,
          **bleu_scores1,
          **bleu_scores2,
          'bleurt_score_a1': bleurt_score_a1,
          'bleurt_score_a2': bleurt_score_a2,
      })

      # Print progress update every 10 samples
      if (idx + 1) % 100 == 0:
          print(f"\nProcessed {idx + 1} samples; Results length: {len(results)}")
          print(f"Latest Answer1 ROUGE-1: {rouge_scores1['rouge1_a1']:.4f}")

  return pd.DataFrame(results)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


In [ ]:
test_im_ds = Dataset.from_list(list(test_im))
test_results_im_df = process_dataset(test_im_ds)

 40%|███▉      | 100/253 [17:43<28:03, 11.00s/it]


Processed 100 samples; Results length: 100
Latest Answer1 ROUGE-1: 0.6667


 79%|███████▉  | 200/253 [35:35<09:02, 10.24s/it]


Processed 200 samples; Results length: 200
Latest Answer1 ROUGE-1: 0.2353


100%|██████████| 253/253 [44:50<00:00, 10.64s/it]


In [ ]:
test_results_im_df.head()

,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,translation_length_a1,reference_length_a1,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bleurt_score_a1,bleurt_score_a2
0,0,why was the cottager treated differently after...,implicit,once upon a time there was a big wedding at a ...,they could see him .,he was received in right friendly fashion .,he was treated differently because he had foun...,0.000000,0.0,0.000000,...,20,5,0.086388,"[0.19047619047619047, 0.1, 0.05263157894736842...",1.0,2.500000,20,8,-1.4250605,-1.0177602
1,1,how did ian feel after he reached the place wh...,implicit,' bear me to the kingdom of the king under the...,nervous .,excited .,"he felt relieved, happy, and contented.",0.000000,0.0,0.000000,...,9,2,0.141140,"[0.2, 0.1111111111111111, 0.125, 0.14285714285...",1.0,4.500000,9,2,-1.6936493,-0.41272795
2,2,how did maie feel after she woke up from her d...,implicit,"' heavens , mother , ' said he , ' where are y...",confused .,very confused .,she felt tired and sleepy.,0.000000,0.0,0.000000,...,6,2,0.220896,"[0.2857142857142857, 0.16666666666666666, 0.2,...",1.0,2.000000,6,3,-1.0883904,-0.9051543
3,3,why was the happy hunter happy to meet the dau...,implicit,""" are you indeed the daughters of ryn jin , th...",he needed to ask ryn jin to help him .,he was looking for ryn jin .,the happy hunter was happy because they had fo...,0.076923,0.0,0.076923,...,19,10,0.071270,"[0.15, 0.05263157894736842, 0.0555555555555555...",1.0,2.714286,19,7,-1.3472155,-1.5319769
4,4,why did the disagreeable neighbor ask the good...,implicit,"he heard at once , through the servant , of hi...",he also wants to remove his wen .,he wanted to know how the old man removed it .,the disagreement between them caused the disag...,0.074074,0.0,0.074074,...,21,8,0.073053,"[0.22727272727272727, 0.047619047619047616, 0....",1.0,1.909091,21,11,-1.3652456,-1.0848835


In [ ]:
test_results_im_df.tail(10)

,id,question,ex_or_im,story_section,reference_answer1,reference_answer2,generated_answer,rouge1_a1,rouge2_a1,rougeL_a1,...,translation_length_a1,reference_length_a1,bleu_a2,precisions_a2,brevity_penalty_a2,length_ratio_a2,translation_length_a2,reference_length_a2,bleurt_score_a1,bleurt_score_a2
243,243,why did ian fall into a dark slumber ?,implicit,"' here i must leave you , ' said the deer , ' ...",he touched an enchanted stick .,the spike of hurt .,he fell asleep after being watched by a sad-lo...,0.074074,0.000000,0.074074,...,23,6,0.058565,"[0.125, 0.043478260869565216, 0.04545454545454...",1.0,4.600000,23,5,-1.6743902,-1.6530638
244,244,how did the wife's daughter feel about getting...,implicit,"with a very ill grace the damsel set forth , g...",annoyed .,upset .,the wife's daughter felt angry because she wan...,0.000000,0.000000,0.000000,...,22,2,0.055387,"[0.08695652173913043, 0.045454545454545456, 0....",1.0,11.000000,22,2,-1.3012695,-1.2779516
245,245,why did the deer drop wax from her ear ?,implicit,"' it is the man who is lying under the cask , ...",used to revive the dead man .,the ear wax can revive the dead .,the deer took care of the dead man.,0.428571,0.333333,0.428571,...,9,7,0.211053,"[0.5, 0.2222222222222222, 0.125, 0.14285714285...",1.0,1.125000,9,8,-0.95999885,-1.2636538
246,246,how did the king know the many-furred creature...,implicit,the king danced again with the beautiful maide...,she wore the ring .,he saw the ring he had put on her hand in the ...,the king knew that the many - furred creature ...,0.076923,0.000000,0.076923,...,25,5,0.063948,"[0.23076923076923078, 0.04, 0.0416666666666666...",1.0,1.785714,25,14,-1.574779,-1.410674
247,247,why did the demon want a pledge ?,implicit,"the old man thanked him very humbly : "" i did ...",he wanted to make sure the man will come back ...,so that the old man would return .,he wanted to show his loyalty to the old man b...,0.400000,0.142857,0.333333,...,19,13,0.132648,"[0.3, 0.15789473684210525, 0.1111111111111111,...",1.0,2.375000,19,8,-0.97738683,-0.54824203
248,248,why did the women offer mikoto water ?,implicit,"when the mikoto saw that he was discovered , h...",he was dignified and gracious .,mikoto said he was thirsty .,they wanted him to feel better and be refreshed.,0.142857,0.000000,0.142857,...,10,6,0.126060,"[0.18181818181818182, 0.1, 0.1111111111111111,...",1.0,1.666667,10,6,-0.10412276,-1.212851
249,249,what will happen after harold sees that paul a...,implicit,but that was just the very thing that happened...,he will become jealous .,he will feel jealous .,the couple will be married.,0.222222,0.000000,0.222222,...,6,5,0.244462,"[0.42857142857142855, 0.16666666666666666, 0.2...",1.0,1.200000,6,5,-1.328775,-1.441332
250,250,how did the gentleman feel after he threw away...,implicit,"left to himself , the full horror of his comin...",horrified .,full of horror .,he realized that he had thrown away his sole c...,0.000000,0.000000,0.000000,...,13,2,0.105711,"[0.21428571428571427, 0.07692307692307693, 0.0...",1.0,3.250000,13,4,-1.6023456,-1.643472
251,251,what did the happy hunter initially expect of ...,implicit,"the happy hunter made all the haste he could ,...",he expected it to take longer .,for the journey to be very long .,he hoped to reach the gate quickly and safely.,0.266667,0.000000,0.266667,...,10,7,0.149911,"[0.36363636363636365, 0.1, 0.1111111111111111,...",1.0,1.250000,10,8,-0.45580557,-0.69976085
252,252,what happened after dullhead shared his cake a...,implicit,"when he got to the forest , he too met the lit...",the little man gave dullhead luck because he h...,the cake turned into a fine rich cake .,the little man was filled with happiness and g...,0.277778,0.117647,0.222222,...,21,17,0.064295,"[0.13636363636363635, 0.047619047619047616, 0....",1.0,2.333333,21,9,-0.62861824,-1.4721348


In [ ]:
test_results_im_df.to_csv('drive/MyDrive/266_Danielle_Dylan_final_project/answer_outputs/qwen_finetuned_imOnly_chat.csv',index=False)

In [ ]:
# def generate_output(model, tokenizer, data, **kwargs):

#     all_outputs = []
#     prefix = "Please answer this question: "
#     context = " Context: "

#     input_sentences = [prefix + sample['question'] + context + sample['story_section'] for sample in data]

#     for i in range(int(len(input_sentences)) + 1):
#         if i >= len(input_sentences):
#             break

#         inputs_encoded = tokenizer(input_sentences[i],truncation=True, padding=True, return_tensors='pt')
#         output_ids = model.cuda().generate(input_ids=inputs_encoded['input_ids'].cuda(), **kwargs)
#         generated_sentences = tokenizer.batch_decode(output_ids,
#                                                      skip_special_tokens=True,
#                                                      clean_up_tokenization_spaces=False)

#         all_outputs.extend(generated_sentences)

#     return all_outputs

In [ ]:
# def calculate_eval_metrics(data, model, tokenizer, prefix="", **kwargs):

#     # Translate original texts
#     answers = generate_output(model, tokenizer, data, **kwargs)
#     references = [example['answer1'] for example in data]

#     # Calculate ROUGE scores
#     rouge_results = rouge.compute(predictions=answers, references=references)
#     print('ROUGE: ', rouge_results)

#     # Calculate BLEURT scores
#     bleurt_scores = []
#     for i in range(int(len(answers)) + 1):
#         if i >= len(answers):
#             break

#         with torch.no_grad():
#             scores = bleurt_model(**bleurt_tokenizer(references[i],
#                                                      answers[i],
#                                                      truncation=True,
#                                                      max_length=MAX_SEQUENCE_LENGTH,
#                                                      padding='max_length',
#                                                      return_tensors='pt'))[0].squeeze().numpy()
#             if scores.shape:
#                 bleurt_scores.extend(scores)
#             else:  # Happens when there was only one example in the last batch
#                 bleurt_scores.append(float(scores))

#     print('BLEURT: ', np.mean(bleurt_scores))

#     return answers

In [ ]:
# """
# Playing with the decoder .generate() arguments like num_beams or top_p, etc.
# """

# generate_kwargs = {

#     'num_beams': 4,
#     'do_sample':True,
#     'top_k':200,
#     'top_p':0.95,
#     'temperature':0.4,
#     'no_repeat_ngram_size':4,
#     'max_new_tokens': 500
# }

# test_im_list = list(test_im.take(100))  # or however many you evaluated

# test_answers_im = calculate_eval_metrics(
#     # test_tokenized_im,
#     test_im_list,
#     model_im_llama,
#     tokenizer,
#     batch_size,
#     **generate_kwargs
# )

In [ ]:
# # Print out a sample of outputs to manually review

# for i in range(10):
#     sample_i = random.choice(range(len(test_im_list)))
#     print('Referenced answer:  ', test_im_list[i]['answer1'])
#     print('Generated answer:   ', test_answers_im[i])
#     print()

In [ ]:
# def clear_gpu_memory():
#     import gc
#     import torch

#     gc.collect()
#     torch.cuda.empty_cache()
#     print("Cleared GPU memory.")

# # Usage
# del model_im_cot, tokenizer, trainer
# clear_gpu_memory()


In [ ]:
def generate_qwen_output(model,tokenizer,story,question):


  messages = [
      {"role": "user", "content": f"Context: {story} {prefix}\n\nQuestion: {question}"},
      {"role": "assistant", "content": ""}
  ]
  text = tokenizer.apply_chat_template(
      messages[:-1],
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=50,
      # pad_token_id=tokenizer.eos_token_id
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return response

In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question with one sentence."
context = " Context: "

# Define the preprocessing function

def preprocess_function_qwen_chat(data):
    """Preprocess data using Qwen2 chat format"""

    full_inputs = []
    for question, story, answer in zip(data["question"], data['story_section'], data['answer1']):
        # Format as a conversation
        messages = [
            {"role": "user", "content": f"Context: {story} {prefix}\n\nQuestion: {question}"},
            {"role": "assistant", "content": answer}
        ]

        # Apply chat template
        formatted_text = tokenizer.apply_chat_template(messages, tokenize=False)
        formatted_text = f"User: Context: {story}\n\nQuestion: {question}\nAssistant: {answer}"
        # print(formatted_text)
        full_inputs.append(formatted_text)

    # Tokenize
    tokenized = tokenizer(
        full_inputs,
        max_length=MAX_SEQUENCE_LENGTH,
        truncation=True,
        padding='max_length'
    )

    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized